In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_movie_budgets(max_pages=1):
    """
    Scrapea la lista de películas de 'https://www.the-numbers.com/movie/budgets/all',
    incluyendo la fecha de lanzamiento (Release Date), título (Movie), presupuesto (Budget),
    recaudación en EE.UU. (Domestic Gross) y recaudación mundial (Worldwide Gross).
    """
    base_url = "https://www.the-numbers.com/movie/budgets/all"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/112.0.0.0 Safari/537.36"
        )
    }
    
    all_data = []
    
    for page in range(1, max_pages + 1):
        offset = (page - 1) * 100
        if offset == 0:
            url = base_url
        else:
            url = f"{base_url}/{offset+1}"
        
        print(f"Scrapeando página {page}: {url}")
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Status {response.status_code}. Se detiene la extracción.")
            break
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Busca la primera tabla que encuentre
        table = soup.find("table")
        if not table:
            print("No se encontró la tabla en la página.")
            continue
        
        # Selecciona todas las filas <tr> de esa tabla
        table_rows = table.find_all("tr")
        
        # Si la tabla tiene una fila de cabecera, se salta
        for row in table_rows[1:]:
            cols = row.find_all("td")
            if len(cols) < 6:
                continue
            
            release_date    = cols[1].get_text(strip=True)
            movie           = cols[2].get_text(strip=True)
            budget          = cols[3].get_text(strip=True).replace('\xa0','')
            domestic_gross  = cols[4].get_text(strip=True).replace('\xa0','')
            worldwide_gross = cols[5].get_text(strip=True).replace('\xa0','')
            
            # Limpieza adicional (quitar $ y comas)
            budget = budget.replace('$', '').replace(',', '')
            domestic_gross = domestic_gross.replace('$', '').replace(',', '')
            worldwide_gross = worldwide_gross.replace('$', '').replace(',', '')
            
            all_data.append({
                'Release Date': release_date,
                'Movie': movie,
                'Budget': budget,
                'Domestic Gross': domestic_gross,
                'Worldwide Gross': worldwide_gross
            })

    df = pd.DataFrame(all_data)
    return df


if __name__ == "__main__":
    df_budgets = scrape_movie_budgets(max_pages=1)
    print(df_budgets.head())
    
    df_budgets.to_csv('../data/the_numbers_movie_budgets-gross.csv', index=False)
    print(f"Guardado CSV con {len(df_budgets)} registros.")


Scrapeando página 1: https://www.the-numbers.com/movie/budgets/all
   Release Date                                        Movie     Budget  \
0  Dec 16, 2015         Star Wars Ep. VII: The Force Awakens  533200000   
1  Apr 23, 2019                            Avengers: Endgame  400000000   
2   Dec 9, 2022                     Avatar: The Way of Water  400000000   
3  May 21, 2025      Mission: Impossible—The Final Reckoning  400000000   
4  May 20, 2011  Pirates of the Caribbean: On Stranger Tides  379000000   

  Domestic Gross Worldwide Gross  
0      936662225      2056046835  
1      858373000      2748242781  
2      684075767      2312413227  
3              0               0  
4      241071802      1045713802  
Guardado CSV con 100 registros.
